In [1]:
from gensim import corpora, models
import pandas as pd
import numpy as np
import sys
from collections import Counter
import itertools
from lsi_tagger.model import TagExtractor

%load_ext autoreload
%autoreload 2

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\David\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
df = pd.read_csv('data/h&m_kaggle_products.csv')
df = df[~df.isnull().any(axis=1)]
print(df.shape)
df.head(2)

(105126, 25)


,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.


In [3]:
documents = df['detail_desc'].drop_duplicates().values

te = TagExtractor(word_count_min=2, word_length_min=2, num_lsi_topics=200, 
                  bigram_kwargs={'bigrams_pmi_min_value':1, 'bigrams_min_freq':200})
te.fit(documents)

Filtering tokens: 100%|████████████████████████████████████████████████████████| 43404/43404 [00:12<00:00, 3453.21it/s]
Making bigrams: 43404it [00:01, 41499.68it/s]
Filtering by word count: 100%|████████████████████████████████████████████████| 43404/43404 [00:00<00:00, 58857.55it/s]
2022-04-09 22:09:20 INFO     Training TF-IDF...
2022-04-09 22:09:20 INFO     adding document #0 to Dictionary(0 unique tokens: [])
2022-04-09 22:09:21 INFO     adding document #10000 to Dictionary(1741 unique tokens: ['jersey', 'jersey top', 'narrow', 'narrow shoulder', 'shoulder']...)
2022-04-09 22:09:22 INFO     adding document #20000 to Dictionary(1937 unique tokens: ['jersey', 'jersey top', 'narrow', 'narrow shoulder', 'shoulder']...)
2022-04-09 22:09:22 INFO     adding document #30000 to Dictionary(2069 unique tokens: ['jersey', 'jersey top', 'narrow', 'narrow shoulder', 'shoulder']...)
2022-04-09 22:09:23 INFO     adding document #40000 to Dictionary(2180 unique tokens: ['jersey', 'jersey top', 'nar

In [4]:
len(te.tc.bigrams_dict)

212

In [5]:
input_tags, candidate_tags = te.transform(input_document='Jersey top with narrow shoulder straps.',
                                          candidate_documents=[
                                              'Tights with built-in support to lift the bottom. Black in 30 denier and light amber in 15 denier.',
                                              'Semi shiny tights that shape the tummy, thighs and calves while also encouraging blood circulation in the legs. Elasticated waist.',
                                              'Jersey top with narrow shoulder straps.',
                                              'uygnhjkl'
                                          ], 
                                          n_input_tags=10, n_candidate_tags=10)

input_tags, candidate_tags

([('jersey top', 1),
  ('narrow shoulder', 1),
  ('narrow', 1),
  ('shoulder straps', 1),
  ('straps', 1),
  ('shoulder', 1),
  ('top', 1),
  ('jersey', 1)],
 [{},
  {},
  {'jersey top': 0.8549823742572913,
   'narrow shoulder': 0.8035616500295002,
   'narrow': 0.6223701917320685,
   'shoulder straps': 0.6127424222074097,
   'straps': 0.5812710634781413,
   'shoulder': 0.566411030702969,
   'top': 0.37643360773759155,
   'jersey': 0.36462090012732523},
  {}])

In [6]:
new_candidate_ranking = te.rank(candidate_tags=candidate_tags, selected_tags=['jersey'])
new_candidate_ranking

[2, 0, 1, 3]

In [7]:
te.save()
te = TagExtractor()
te.load()

In [8]:
input_tags, candidate_tags = te.transform(input_document='Jersey top with narrow shoulder straps.',
                                          candidate_documents=[
                                              'Tights with built-in support to lift the bottom. Black in 30 denier and light amber in 15 denier.',
                                              'Semi shiny tights that shape the tummy, thighs and calves while also encouraging blood circulation in the legs. Elasticated waist.',
                                              'Jersey top with narrow shoulder straps.',
                                              'uygnhjkl'
                                          ], 
                                          n_input_tags=10, n_candidate_tags=10)

input_tags, candidate_tags

([('jersey top', 1),
  ('narrow shoulder', 1),
  ('shoulder straps', 1),
  ('straps', 1),
  ('narrow', 1),
  ('shoulder', 1),
  ('top', 1),
  ('jersey', 1)],
 [{},
  {},
  {'jersey top': 0.8772317763823081,
   'narrow shoulder': 0.7785568909779876,
   'shoulder straps': 0.6355535130824466,
   'straps': 0.6059983392044233,
   'narrow': 0.6053735593033779,
   'shoulder': 0.5555633523654915,
   'top': 0.36434324889827213,
   'jersey': 0.35461624727365776},
  {}])

In [9]:
new_candidate_ranking = te.rank(candidate_tags=candidate_tags, selected_tags=['jersey'])
new_candidate_ranking

[2, 0, 1, 3]

In [107]:
# # TODO: Can give the user options of adjacent tags so that they can get new queries entirely?
# # They can remove tags from the input document and add in other tags, like swapping out pants for jeans, etc.
# # This results in a "new" input document

# lsi_topic_matrix_T_normed = te.lsi_topic_matrix.T/np.linalg.norm(te.lsi_topic_matrix.T, axis=1)[:,None]

# def lsi_corpus2vec(lsi_corpus_vec):
#     vec = np.zeros(200)
#     for ind, value in lsi_corpus_vec:
#         vec[ind] = value
#     return vec

# tags = te.tc.transform([df['detail_desc'].drop_duplicates().values[779]])[0]
# print(tags)
# bow = te.dictionary.doc2bow(tags)
# bow_vec = lsi_corpus2vec(te.lsi_model[te.tfidf[bow]])
# bow_vec_normed = bow_vec/np.linalg.norm(bow_vec)
# sims = bow_vec_normed.dot(lsi_topic_matrix_T_normed.T)
# top_inds = np.argsort(-sims)
# count = 0
# for t in top_inds:
#     tag = te.dictionary[t]
#     if tag not in tags:
#         print(tag)
#         count += 1
#     if count == 5:
#         break

In [30]:
def lsi_corpus2vec(lsi_corpus_vec):
    vec = np.zeros(te.num_lsi_topics)
    for ind, value in lsi_corpus_vec:
        vec[ind] = value
    return vec

lsi_vecs = np.array([lsi_corpus2vec(cl) for cl in te.corpus_lsi])
lsi_vecs_normed = lsi_vecs/np.linalg.norm(lsi_vecs, axis=1)[:,None]

In [102]:
lsi_topic_matrix_T_normed = te.lsi_topic_matrix.T/np.linalg.norm(te.lsi_topic_matrix.T, axis=1)[:,None]

def adjacent_tags(input_tags, n_adjacent_tags=10):
    bow = te.dictionary.doc2bow(input_tags)
    bow_vec = lsi_corpus2vec(te.lsi_model[te.tfidf[bow]])
    bow_vec_normed = bow_vec/np.linalg.norm(bow_vec)
    sims = bow_vec_normed.dot(lsi_topic_matrix_T_normed.T)
    top_inds = np.argsort(sims)[::-1]
    
    adj_tags = []
    count = 0
    for t in top_inds:
        tag = te.dictionary[t]
        if tag not in input_tags:
            count += 1
            adj_tags.append(tag)
        if count == n_adjacent_tags:
            break
    return adj_tags

In [105]:
def show_recommendations_with_tags(input_document, 
                                   internal_n_recs=50, show_n_recs=5,
                                   n_input_tags=10, n_candidate_tags=5,
                                   n_adjacent_tags=10,
                                   selected_tags=[]):
    _, input_corpus_lsi = te._get_vector_representations(input_document)
    input_lsi_vec = lsi_corpus2vec(input_corpus_lsi)
    input_lsi_vec_normed = input_lsi_vec/np.linalg.norm(input_lsi_vec)
    sims = input_lsi_vec_normed.dot(lsi_vecs_normed.T)
    top_inds = np.argsort(sims)[::-1][:(internal_n_recs+1)]
    candidate_documents = [documents[ti] for ti in top_inds if documents[ti]!=input_document]
    input_tags, candidate_tags = te.transform(input_document=input_document,
                                              candidate_documents=candidate_documents, 
                                              n_input_tags=n_input_tags, 
                                              n_candidate_tags=n_candidate_tags)
    input_adjacent_tags = adjacent_tags(input_tags=[t[0] for t in input_tags],
                                        n_adjacent_tags=n_adjacent_tags)
    if len(selected_tags) > 0:
        ranking = te.rank(candidate_tags=candidate_tags, 
                          selected_tags=selected_tags)
        candidate_documents = np.array(candidate_documents)[ranking]
        candidate_tags = np.array(candidate_tags)[ranking]

    print(input_document)
    print(f"Input Tags: {input_tags}")
    print(f"Adjacent Tags: {input_adjacent_tags}")
    print('='*150)
    print()

    for n, candidate_doc in enumerate(candidate_documents):
        print(candidate_doc)
        display(candidate_tags[n])
        print('-'*150)
        if (n+1) == show_n_recs:
            break

In [ ]:
# Adjacent tags are for adding tokens/tags or replacing them to create a new input "document" to get new recommendations and pairs of tags

In [106]:
show_recommendations_with_tags(input_document=documents[0], 
                               internal_n_recs=500, show_n_recs=5,
                               n_input_tags=10, n_candidate_tags=5, 
                               n_adjacent_tags=10,
                               selected_tags=[])

Jersey top with narrow shoulder straps.
Input Tags: [('shoulder straps', 486), ('straps', 486), ('narrow', 478), ('shoulder', 473), ('narrow shoulder', 450), ('top', 39), ('jersey top', 36), ('jersey', 25)]
Adjacent Tags: ['ballon', 'join', 'swimsuits', 'knots', 'cross', 'sock', 'bodies', 'narrow cut', 'tankini', 'bustier']

Fitted jersey top with narrow shoulder straps.


{'jersey top': 0.7778493927084533,
 'narrow shoulder': 0.7443624326659819,
 'shoulder straps': 0.6084045654463418,
 'narrow': 0.5992542723284608,
 'straps': 0.5735725438985558}

------------------------------------------------------------------------------------------------------------------------------------------------------
Jersey top with a V-neck and narrow shoulder straps.


{'jersey top': 0.7895180995988038,
 'narrow shoulder': 0.7424408172030121,
 'shoulder straps': 0.6128653136562148,
 'narrow': 0.5997496712896061,
 'straps': 0.5798600112898259}

------------------------------------------------------------------------------------------------------------------------------------------------------
Short, fitted jersey top with narrow shoulder straps.


...

------------------------------------------------------------------------------------------------------------------------------------------------------
Pleated jersey top with a metallic coating, narrow shoulder straps and a V-neck at the front.


{'jersey top': 0.6874856961237855,
 'narrow shoulder': 0.635401918399671,
 'shoulder straps': 0.5243586911490498,
 'narrow': 0.5180327519143767,
 'straps': 0.4970202451085512}

------------------------------------------------------------------------------------------------------------------------------------------------------
Jersey top in a soft silk and cotton blend with narrow shoulder straps.


{'jersey top': 0.7100134061329838,
 'narrow shoulder': 0.6886866103953119,
 'shoulder straps': 0.5609481587147457,
 'narrow': 0.539734304551692,
 'straps': 0.5325082234881369}

------------------------------------------------------------------------------------------------------------------------------------------------------
